# Projeto 1 - Ciência dos Dados

Nome: Rafael Melhado Araujo Lima

Nome: Victor Pacheco Santos

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    #ACHAR UM JEITO DE RETIRAR OS @S E OS NOMES QUE VEM SEGUIDOS DELES
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed


In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
/Users/mac/Desktop/ciencia-dos-dados-p1-main 6


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [23]:
filename = 'iFood.xlsx'

In [24]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Classificação
0,me dá cupom de desconto @ifood,1
1,é hoje!\na partir das 18:30 estaremos 100% onl...,0
2,todo dia eu entro no ifood pra ver se ele tá m...,0
3,eu n to acreditando que o entregador do ifood ...,2
4,preciso apresentar essa música p uns parceiro ...,0


In [28]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head()

,Teste
0,@matheusdevdd @rclorandi @rolealeatorio @ifood...
1,@taldoalvesss eu fico parecendo um entregador ...
2,@gdallaru @ifood acordou engraçadinho e com a ...
3,"pedido mais rápido q já fiz no ifood, 20 min f..."
4,"ele não pega ninguém, só leva p dar rolê, ifoo..."


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O produto escolhido foi o serviço do iFood. Para a classificação dos tweets, consideramos relevantes aqueles que forneciam opiniões de clientes, com claras críticas ou elogios. Além disso, classificamos os tweets relevantes como sendo ou positivos ou negativos, para que foçe mais fácil a separação de críticas e elogios.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [29]:
Limpo = []
for i in range (len(train.Treinamento)):
    Limpo.append(cleanup(train.Treinamento[i].lower()))
train["Limpo"] = pd.Series(Limpo)

tudo = ""
for tweet in train.Limpo:
    tudo = tudo + tweet
tudo_separado = tudo.split()

In [101]:
positiva = train.loc[(train.Classificação == 1), "Limpo"]
negativa = train.loc[(train.Classificação == 2), "Limpo"]
irrelevante = train.loc[(train.Classificação == 0), "Limpo"]

positiva_texto = ""
for tweet in positiva:
    positiva_texto = positiva_texto + tweet

negativa_texto = ""
for tweet in negativa:
    negativa_texto = negativa_texto + tweet

    
irrelevante_texto = ""
for tweet in irrelevante:
    irrelevante_texto = irrelevante_texto + tweet
    


positiva_separado = positiva_texto.split()
negativa_separado = negativa_texto.split()
irrelevante_separado = irrelevante_texto.split()


tudo_junto = (positiva_separado+negativa_separado+irrelevante_separado)
prob_p = len(positiva_separado)/len(tudo_separado)
prob_n = len(negativa_separado)/len(tudo_separado)
prob_i = len(irrelevante_separado)/len(tudo_separado)


In [102]:
serie_positiva = pd.Series(positiva_separado)
tabela_positiva = serie_positiva.value_counts()
tabela_positiva_relativa = serie_positiva.value_counts(True)

serie_irrelevante = pd.Series(irrelevante_separado)
tabela_irrelevante = serie_irrelevante.value_counts()
tabela_irrelevante_relativa = serie_irrelevante.value_counts(True)

serie_negativa = pd.Series(negativa_separado)
tabela_negativa = serie_negativa.value_counts()
tabela_negativa_relativa = serie_negativa.value_counts(True)

serie_junto = pd.Series(tudo_junto)
tabela_junto = serie_junto.value_counts()

In [110]:
def classifica_tweets(tweet):
    ProbP = prob_p
    ProbN = prob_n
    ProbI = prob_i
    probTweetDadoP = 1
    probTweetDadoN = 1
    probTweetDadoI = 1
    tweet_separado = tweet.split()
    for i in range(len(tweet_separado)):
        if tweet_separado[i] in tabela_positiva_relativa:
            probTweetDadoP = probTweetDadoP*(tabela_positiva[tweet_separado[i]]+1)/(len(serie_positiva) + len(tabela_junto))
        else:
            probTweetDadoP = probTweetDadoP*(0+1)/(len(serie_positiva) + len(tabela_junto))
        if tweet_separado[i] in tabela_negativa_relativa:
            probTweetDadoN = probTweetDadoN*(tabela_negativa[tweet_separado[i]]+1)/(len(serie_negativa) + len(tabela_junto))
        else:
            probTweetDadoN = probTweetDadoN*(0+1)/(len(serie_negativa) + len(tabela_junto))
        if tweet_separado[i] in tabela_irrelevante_relativa:
            probTweetDadoI = probTweetDadoI*(tabela_irrelevante[tweet_separado[i]]+1)/(len(serie_irrelevante) + len(tabela_junto))
        else:
            probTweetDadoI = probTweetDadoI*(0+1)/(len(serie_irrelevante) + len(tabela_junto))
    ProbPDadoTweet = probTweetDadoP*ProbP
    print(ProbPDadoTweet)
    ProbNDadoTweet = probTweetDadoN*ProbN
    print(ProbNDadoTweet)
    ProbIDadoTweet = probTweetDadoI*ProbI
    print(ProbIDadoTweet)
    if ProbPDadoTweet > ProbIDadoTweet and ProbPDadoTweet > ProbNDadoTweet:
        return("O tweet '{0}' é positivo".format(tweet))
    elif ProbNDadoTweet > ProbIDadoTweet and ProbNDadoTweet > ProbPDadoTweet:
        return("O tweet '{0}' é negativo".format(tweet))
    else:
        return("O tweet '{0}' é irrelevante".format(tweet))
    
classifica_tweets("pedido mais rápido q já fiz no ifood 20 min foi o tempo entre eu fazer o pedido e abrir a marmita exatos 20 min")

2.5090001156205487e-73
4.9571114945964685e-73
7.338212646428386e-70


"O tweet 'pedido mais rápido q já fiz no ifood 20 min foi o tempo entre eu fazer o pedido e abrir a marmita exatos 20 min' é irrelevante"

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [109]:
teste_limpo = []

for i in range (len(test.Teste)):
    teste_limpo.append(cleanup(test.Teste[i].lower()))
    
for tweet in teste_limpo:
    print(classifica_tweets(tweet))

1.3559859844750633e-33
7.98754103798418e-33
1.1818651182089633e-31
O tweet '@matheusdevdd @rclorandi @rolealeatorio @ifood tirei do seu cuzinho inclusive arrombado' é irrelevante
5.880942573985613e-23
8.972101248079938e-24
2.7716348007485087e-23
O tweet '@taldoalvesss eu fico parecendo um entregador do ifood' é positivo
2.4346864003574575e-54
6.726044326754798e-52
7.203707647429031e-54
O tweet '@gdallaru @ifood acordou engraçadinho e com a foto desse ombro com marrinha de porra nenhuma né @victorratz' é negativo
2.5090001156205487e-73
4.9571114945964685e-73
7.338212646428386e-70
O tweet 'pedido mais rápido q já fiz no ifood 20 min foi o tempo entre eu fazer o pedido e abrir a marmita exatos 20 min' é irrelevante
2.49007148242981e-44
3.1821235071057738e-46
1.136965455319652e-43
O tweet 'ele não pega ninguém só leva p dar rolê ifood ifood de mulher kkkkkkkkkk 🤌🏾' é irrelevante
3.365317149124482e-38
6.1913488556474404e-40
3.4441676549909067e-38
O tweet 'o menino pedindo pra eu mandar um i

1.656057405735617e-41
2.4282033490269734e-43
3.503038061576945e-41
O tweet 'cupom de 10 reais do ifood eu te amo comprei meu almoço por 3 reais' é irrelevante
9.121772052176006e-90
4.527205812799821e-84
4.1892541886757214e-88
O tweet 'como vocês acham q tá a economia do país e da população quando autônomos começam a fazer paralisação e greve 

@brasil  @governopara @belemtransito 

negociação barganha caminhoneiro uber ifood' é negativo
1.2663782439587358e-98
6.132340478698317e-98
1.1665598899865877e-94
O tweet 'paguei minhas contas e já to aq pensando na próxima fatura vou entrar em total estado de economia uber em último caso ifood excluído com sucesso e sem compras online compulsivas' é irrelevante
3.353982486868868e-85
2.6546190777751157e-87
1.082917869158211e-87
O tweet 'poucas coisas me deixam mais puto que quando o ifood manda o entregador mais longe que o restaurante com outro pedido junto do meu

a entrega demora o triplo do tempo' é positivo
1.5456966017137381e-22
8.471553603

In [68]:
v_p = 1
f_p = 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
v_n = 1 1 1 1 1 1 1 1 1
f_n = 1 1 1 1 
v_i = 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
f_i = 1 1 1 1 1 1 1 1 1

42
35
51
30
35
7
200
0.64


___
### Concluindo

In [ ]:
#Em sumo, foi podemos concluir que o classificador funciona de maneira adequada. Porém, ele não é completamente
#preciso, tendo uma acuracia de aproximadamente 64%. Além disso, quando analisamos os tweets erroneamente 
#classificados como positivos, vemos a menor precisão. Um dos possíveis motivos para tal discrepância, 
#além do tamanho da amostra inicial, é que o classificador não consegue distinguir diferentes contextos,
#logo um tweet extremamente sarcástico, pode ser classificado como positivo, quando na verdade é negativo, e 
#vice-e-versa, o que exarceba a falta de precisão do classificador. 

#Alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets,
#provavelmente causará grandes problemas, pois o fato de sua acuracia ser relativamente baixa fará com que
#sejam alimentados muitos dados incorretos, o que propagará vícios do sistema, fazendo com que o classificador se 
#torne cada vez mais impreciso.

#Possiveis melhorias para o classificador incluem um aumento na base de dados à ele alimentados. Além disso, uma
#classificação mais Outra possível 
#melhoria para o classificador poderia ser o uso de outro método de classificação 

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**